In [74]:
import os
import math
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.special import comb
from scipy.stats import hypergeom
from matplotlib import rcParams
from collections import defaultdict
from pyCRAC.Methods import sortbyvalue
rcParams['font.family'] = 'sans-serif'
rcParams['font.sans-serif'] = ['Arial']
rcParams['pdf.fonttype'] = 42
rcParams['axes.formatter.useoffset'] = False

In [21]:
data = pd.read_csv('../../../big_dataframe_Xist_diffBUM_HMM_scaled_dc_deltaSHAPE_corrected.txt',\
                    sep="\t",\
                    header=0,\
                    index_col=0)

In [22]:
data.columns

Index(['nucleotide_number', 'nucleotide_identity', 'deltaSHAPE_rep1',
       'deltaSHAPE_rep2', 'average_deltaSHAPE', 'CELF1', 'FUS', 'HuR', 'PTBP1',
       'RBFOX2', 'TARDBP', 'diffBUMHMM_in_cell', 'diffBUMHMM_ex_vivo',
       'dStruct_DDR'],
      dtype='object')

In [23]:
in_cell = data[data.diffBUMHMM_in_cell > 0.95]["nucleotide_identity"].values
ex_vivo = data[data.diffBUMHMM_ex_vivo > 0.95]["nucleotide_identity"].values

In [24]:
nucdict = defaultdict(int)
for nuc in in_cell:
    nucdict[nuc] += 1

In [25]:
nucdict

defaultdict(int, {'C': 36, 'T': 54, 'A': 49, 'G': 49})

In [26]:
nucdict = defaultdict(int)
for nuc in ex_vivo:
    nucdict[nuc] += 1

In [27]:
nucdict

defaultdict(int, {'C': 374, 'A': 466, 'G': 352, 'T': 547})

In [28]:
ex_vivo_pos = data[data.diffBUMHMM_ex_vivo > 0.95]["nucleotide_number"].values

In [29]:
ex_vivo_pos

array([   63,    70,    72, ..., 17696, 17697, 17699])

In [39]:
sequence = "".join(data.nucleotide_identity.values)

In [40]:
sequence[:20]

'CGGCTTGCTCCAGCCATGTT'

In [117]:
def calcMotifZscores(actual,random):
    """Calculates the Z-scores for each k-mer sequence"""
    motif_list = list()
    z_scores = defaultdict(float)
    totalmotifs = len(actual)
    actualcount = defaultdict(int)
    for i in actual:
        actualcount[i] += 1
    randomcount = defaultdict(int)
    for i in random:
        randomcount[i] += 1
    a = set(actual)
    b = set(random)
    motif_list = a | b
    for motifs in motif_list:
        p1 = float(actualcount[motifs]) / totalmotifs
        p2 = float(randomcount[motifs]) / totalmotifs
        try:
            if p1 or p2:
                score = math.sqrt(totalmotifs) * (p1-p2) / math.sqrt(p1*(1-p1) + p2*(1-p2))
                z_scores[motifs] = score
        except ValueError:
            sys.stderr.write("\nValueError! Can not calculate a Z-score for the %s motif. The number of motifs (%s) is higher than the total number of sequences (%s)\n" % (motifs,self.__motif_count[motifs],self.total_motif_count))
            pass
    return z_scores

In [118]:
def getKmers(positions,sequence,length=4):
    assert length % 2 == 0, "can only process k-mers with even numbers!"
    k_mers = list()
    for i in positions:
        start = int(i-(length/2))
        end = int(i+(length/2))
        if start < 0:
            start = 0 ### in case the start position is a negative value
            end = length+1
        k_mers.append(sequence[start:end])
    return k_mers

In [119]:
randpos = np.random.choice(np.arange(len(sequence)),len(ex_vivo_pos))

In [120]:
randpos

array([14920,  3688,   307, ..., 11512,    62,  2702])

In [121]:
actual = getKmers(ex_vivo_pos,sequence,6)

In [122]:
random = getKmers(randpos,sequence,6)

In [123]:
scores = calcMotifZscores(actual,random)

In [124]:
for motif,score in sortbyvalue(scores):
    print("%s\t%s" % (motif,score))

TTCCTT	2.2392895149396312
AATATT	2.2392895149396312
TGTGTC	2.0023041482295567
ACAAAG	2.0023041482295567
AATAAA	2.0023041482295567
TTGCTT	1.7335467492390375
TGGCCT	1.7335467492390375
TCTCTT	1.7335467492390375
TCCTAA	1.7335467492390375
TACCCT	1.7335467492390375
TAATTT	1.7335467492390375
GTGTTT	1.7335467492390375
GTGCCC	1.7335467492390375
GCAGTC	1.7335467492390375
CTTTAG	1.7335467492390375
CTTCCA	1.7335467492390375
CCTTCC	1.7335467492390375
CATTAT	1.7335467492390375
CAGGCT	1.7335467492390375
CACAGT	1.7335467492390375
CAAAGA	1.7335467492390375
ATTCTT	1.7335467492390375
ATATAA	1.7335467492390375
AGTGTT	1.7335467492390375
AGCCTT	1.7335467492390375
ACTGTG	1.7335467492390375
AAAGCA	1.7335467492390375
TTTTGG	1.4150274983037154
TTTGGC	1.4150274983037154
TTTGAC	1.4150274983037154
TTTAGC	1.4150274983037154
TTGAAC	1.4150274983037154
TTCCAA	1.4150274983037154
TTATAT	1.4150274983037154
TTACCC	1.4150274983037154
TTAAGG	1.4150274983037154
TTAACT	1.4150274983037154
TTAAAA	1.4150274983037154
TGTTTC	1.415